In [45]:
import math
import json
import re
import html
from urllib.request import Request, urlopen
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import numpy as np
import seaborn as sns
%matplotlib inline
sns.set_style("whitegrid")

df
for i in range(0,1):
    request=Request('https://public-api.wordpress.com/wp/v2/sites/techcrunch.com/posts?offset='+str(i*100)+'&per_page=100')
    response = urlopen(request).read().decode('utf8')
    df = df.append(pd.io.json.json_normalize(json.loads(response)))

# remove unneeded cols
df = df.drop('_links.about', 1).drop('_links.author', 1).drop('_links.collection', 1).drop('_links.https://api.w.org/attachment', 1).drop('_links.https://api.w.org/featuredmedia', 1).drop('_links.https://api.w.org/meta', 1).drop('_links.https://api.w.org/term', 1).drop('_links.replies', 1).drop('_links.self', 1)
df = df.drop('_links.version-history', 1).drop('comment_status',1).drop('featured_media',1).drop('format',1).drop('guid.rendered',1).drop('modified',1).drop('date_gmt',1).drop('excerpt.rendered',1).drop('modified_gmt',1).drop('ping_status',1).drop('sticky',1)
df = df[df['type'] == "post"]
df = df.drop('type', 1)

# clean html
df['title.rendered'] = df['title.rendered'].apply(lambda x: html.unescape(x)) 
df['content.rendered'] = df['content.rendered'].apply(lambda x: html.unescape(x))

def remove_html(x):
    clean = re.sub(re.compile('<.*?>'), '', x) 
    clean = re.sub(re.compile('\[.*?]'), '', clean)
    clean = clean.rstrip().lstrip()
    return clean

df['content.rendered'] = df['content.rendered'].apply(remove_html)

In [46]:
# feature engineering
df['views'] =  1
df['views'] = df['views'].apply(lambda x: np.random.randint(10000, 500000)) 

# dist = np.random.normal(250000, 0.2, 10000)
# df['views'] = df['views'].applymap(lambda x: dist)  # update size according to dataset
# sns.distplot(dist)

df.sample(5)

,author,categories,content.rendered,date,id,link,slug,tags,title.rendered,views
53,12084649,"[1559, 17396]",There was one slide in Tim Cook’s keynote that...,2016-09-08T19:26:04,1382617,https://techcrunch.com/2016/09/08/the-swiss-wa...,the-swiss-watch-industry-is-doomed,[1055781],The Swiss watch industry is doomed,410461
45,31035538,"[173267257, 20429, 17396]","Beeswax, a startup founded by a trio of former...",2016-09-09T07:11:36,1382717,https://techcrunch.com/2016/09/09/beeswax-seri...,beeswax-series-a,"[449553193, 2227692, 10917493]",Beeswax raises $11M for customizable adtech,494925
36,88203759,"[15835174, 17396]",Here’s a cool feature from Square Cash – the c...,2016-09-09T10:38:57,1382814,https://techcrunch.com/2016/09/09/square-cash-...,square-cash-users-can-now-spend-their-balance-...,"[43029, 67376, 9881, 216504, 63860, 171190, 17...",Square Cash users can now spend their balance ...,452999
56,170861,"[449223024, 20429, 17396]",Most startups toil away in stealth mode on new...,2016-09-08T16:46:59,1370981,https://techcrunch.com/2016/09/08/peak-design/,peak-design,"[609795, 344761, 449553141]",Peak Design and building a company on the Kick...,455184
9,12084649,"[1559, 17396]","While the Camorama promo video is a bit, shall...",2016-09-10T09:22:28,1383173,https://techcrunch.com/2016/09/10/camorama-rec...,camorama-records-360-degree-4k-video-for-your-...,"[419789908, 860595, 245726669, 412, 18949, 6567]",Camorama records 360-degree 4K video for your ...,288021


In [47]:
file_ = open('articles.json', 'w')
file_.write(df.reset_index().to_json(orient='records'))
file_.close()

In [48]:
highest_views = df.nlargest(math.ceil(len(df.index) * .1), 'views')
lowest_views = df.nsmallest(math.ceil(len(df.index) * .1), 'views')

file_ = open('highest_viewed_articles.json', 'w')
file_.write(highest_views.reset_index().to_json(orient='records'))
file_.close()

file_ = open('lowest_viewed_articles.json', 'w')
file_.write(lowest_views.reset_index().to_json(orient='records'))
file_.close()

In [53]:
lowest_views.sample(5)

,author,categories,content.rendered,date,id,link,slug,tags,title.rendered,views
80,2414667,"[3457, 20429, 17396]",Need to post content to multiple social networ...,2016-09-08T09:07:23,1382286,https://techcrunch.com/2016/09/08/yalas-new-ch...,yalas-new-chatbot-knows-the-best-time-to-post-...,"[175, 49818]",Yala’s new chatbot knows the best time to post...,22487
47,39990176,"[1342, 4328, 449223024, 17396]",The Raspberry Pi Foundation has finally put ou...,2016-09-09T04:54:44,1382701,https://techcrunch.com/2016/09/09/raspberry-pi...,raspberry-pi-finally-offers-an-official-starte...,"[1382984, 2428084, 1896567, 49695267, 56661568]",Raspberry Pi finally offers an official starte...,14896
30,2414667,"[4213, 17396]","Too lazy to read your tweets? Now, Amazon Alex...",2016-09-09T13:14:24,1382962,https://techcrunch.com/2016/09/09/twitter-laun...,twitter-launches-an-alexa-app,"[153122, 599182]",Twitter launches an Alexa app,28956
33,89646803,[17396],"The Honest Company, the consumer products star...",2016-09-09T11:55:00,1382820,https://techcrunch.com/2016/09/09/confirmed-ho...,confirmed-honest-company-in-talks-to-sell,"[33663, 977, 6877, 71123637]",Confirmed: Honest Company in talks to sell,12181
71,8102444,[17396],"Following yesterday’s iPhone 7 announcement, A...",2016-09-08T11:32:01,1382430,https://techcrunch.com/2016/09/08/its-now-only...,its-now-only-29-to-fix-your-cracked-iphone-if-...,"[291, 43124, 106595, 120]",It’s now only $29 to fix your cracked iPhone i...,39169
